# 入门
### Hello World

In [ ]:
import tensorflow as tf
c = tf.constant('hello')
with tf.Session() as sess:
    print(sess.run(c))

### add与+的区别？
实际上最终调用是一个函数，都到了gen_math_ops.add。
tf.add直接调用库方法，+是重载了__add__符号，在tensorflow/python/ops/math_ops.py中定义了_OverrideBinaryOperatorHelper(gen_math_ops.add, "add")，也就是说最终还是调用了tf.add

加载模块tensorflow/python/ops/math_ops.py
```
tensorflow/python/ops/math_ops.py： _OverrideBinaryOperatorHelper(gen_math_ops.add, "add") 
-> tensorflow/python/ops/math_ops.py： clazz_object._override_operator("__%s__" % op_name, binary_op_wrapper) 在binary_op_wrapper中会实际调用gen_math_ops.add
-> tensorflow/python/framework/ops.py： _override_helper(Tensor, operator, func)
-> tensorflow/python/framework/ops.py： setattr(clazz_object, operator, func)
```

* Python 中的setattr有点像Ruby元编程中的动态方法，既可以是属性值，也可以是函数方法。setattr(obj, '__add__', add)也就是动态重载+运算符*

所有重载的运算符在Tensor.OVERLOADABLE_OPERATORS:中进行了定义。

参考：
https://stackoverflow.com/questions/37900780/in-tensorflow-what-is-the-difference-between-tf-add-and-operator

In [20]:
import tensorflow as tf
a = tf.constant(1)
b = tf.constant(2)
with tf.Session() as sess:
    print(sess.run(a+b))
with tf.Session() as sess:
    print(sess.run(tf.add(a,b)))

3
3


### 一个简单的线性模型

In [28]:
import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float16(np.random.rand(100) * 10) # 随机输入
y_data = x_data * 3.2 + 5.1

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([1]))
y = x_data * W + b

# 最小化方差
#loss = tf.reduce_mean(tf.square(y - y_data))
loss = tf.reduce_sum(tf.pow(y_data - y, 2))
train = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 3000):
    sess.run(train)
    if step % 100 == 0:
        print(step, sess.run(loss), sess.run(W), sess.run(b))

0 1.85624e+06 [ 26.77856827] [ 4.27478123]
100 nan [ nan] [ nan]
200 nan [ nan] [ nan]
300 nan [ nan] [ nan]
400 nan [ nan] [ nan]
500 nan [ nan] [ nan]
600 nan [ nan] [ nan]
700 nan [ nan] [ nan]
800 nan [ nan] [ nan]
900 nan [ nan] [ nan]
1000 nan [ nan] [ nan]
1100 nan [ nan] [ nan]
1200 nan [ nan] [ nan]
1300 nan [ nan] [ nan]
1400 nan [ nan] [ nan]
1500 nan [ nan] [ nan]
1600 nan [ nan] [ nan]
1700 nan [ nan] [ nan]
1800 nan [ nan] [ nan]
1900 nan [ nan] [ nan]
2000 nan [ nan] [ nan]
2100 nan [ nan] [ nan]
2200 nan [ nan] [ nan]
2300 nan [ nan] [ nan]
2400 nan [ nan] [ nan]
2500 nan [ nan] [ nan]
2600 nan [ nan] [ nan]
2700 nan [ nan] [ nan]
2800 nan [ nan] [ nan]
2900 nan [ nan] [ nan]
